參考資料 : https://github.com/andy6804tw/2021-13th-ironman/tree/main/4.%E5%92%B1%E5%80%91%E4%B8%80%E8%B5%B7%E5%81%9A%E8%B3%87%E6%96%99%E6%B8%85%E7%90%86%E5%92%8C%E5%89%8D%E8%99%95%E7%90%86

#### 前言
很多演算法對數據範圍非常的敏感。因此為了要讓模型訓練的更強大，通常的做法是對特徵進行調節，使得數據更適合這些演算法。一般來說，我們在做機器學習時往往會做特徵的正規化。

#### 載入相關套件

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # 繪圖套件
import seaborn as sns # 以 matplotlib 為底層的高階繪圖套件
from sklearn.datasets import load_iris # data

np.set_printoptions(suppress=True)  # 全域設定, 限制陣列列印元素方式或數量,此為是否抑制顯示小數位, 參考"基本指令"

#### 載入資料集

In [2]:
iris = load_iris()
df_data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
df_data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


#### 檢查缺失值
使用 numpy 所提供的函式來檢查是否有 NA 缺失值，假設有缺失值使用 dropna() 來移除。使用的時機在於當只有少量的缺失值適用，若遇到有大量缺失值的情況，或是本身的資料量就很少的情況下建議可以透過機器學習的方法補值來預測缺失值。

In [4]:
print("checked missing data(raw data):",len(np.where(np.isnan(df_data['Species']))))
X = df_data.drop(labels=['Species'],axis=1).values # 移除Species並取得剩下欄位資料
y = df_data['Species']
# checked missing data
print("delete missing data")
print("checked missing data(NAN mount):",len(np.where(np.isnan(X))[0]))

checked missing data(raw data): 1
delete missing data
checked missing data(NAN mount): 0


#### 切割訓練集與測試集
我們透過 Sklearn 所提供的 train_test_split() 方法來為我們的資料進行訓練集與測試集的切割。在此方法中我們可以設定一些參數來讓我們切割的資料更多樣性。其中 test_size 參數就是設定測試集的比例。\
另外預設資料切割的方式是隨機切割 shuffle=True 對原始數據進行隨機抽樣，以保證隨機性。若想要每次程式執行時切割結果都是一樣的可以設定亂數隨機種子 random_state 並給予一個隨機數值。\
最後一個是 stratify 分層隨機抽樣，特別是在原始數據中樣本標籤分佈不均衡時非常有用。使用時機是確保分類問題 y 的類別數量分佈要與原資料集一致。以免資料集切割不平均導致模型訓練時有很大的偏差。

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print('train shape:', X_train.shape)
print('test shape:', X_test.shape)

train shape: (105, 4)
test shape: (45, 4)


#### Standardization平均&變異數標準化
將所有特徵標準化，也就是高斯分佈。使得數據的平均值為 0，方差為 1。適合的使用時機於當有些特徵的方差過大時，使用標準化能夠有效地讓模型快速收斂。

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [7]:
# scaled之後的資料零均值，單位方差  
print('資料集 X 的平均值 : ', X_train.mean(axis=0))
print('資料集 X 的標準差 : ', X_train.std(axis=0))

print('\nStandardScaler 縮放過後訓練集的平均值 : ', X_train_scaled.mean(axis=0))
print('StandardScaler 縮放過後訓練集的標準差 : ', X_train_scaled.std(axis=0))

資料集 X 的平均值 :  [5.87333333 3.0552381  3.7847619  1.20571429]
資料集 X 的標準差 :  [0.85882164 0.45502087 1.77553646 0.77383751]

StandardScaler 縮放過後訓練集的平均值 :  [ 0. -0. -0. -0.]
StandardScaler 縮放過後訓練集的標準差 :  [1. 1. 1. 1.]


訓練集的 Scaler 擬合完成後，我們就能做相同的轉換在測試集上。

In [8]:
X_test_scaled = scaler.transform(X_test)

print('\nStandardScaler 縮放過後測試集的平均值 : ', X_test_scaled.mean(axis=0))
print('StandardScaler 縮放過後測試集的標準差 : ', X_test_scaled.std(axis=0))


StandardScaler 縮放過後測試集的平均值 :  [-0.11643861  0.01534903 -0.05024191 -0.02748619]
StandardScaler 縮放過後測試集的標準差 :  [0.85754489 0.83947065 0.96847064 0.9374037 ]


如果想將轉換後的資料還原可以使用 inverse_transform() 將數值還原成原本的輸入。

In [12]:
# 將縮放的資料還原
X_test_inverse = scaler.inverse_transform(X_test_scaled)
print("原始資料 : \n",X_test[:3])
print("標準化再還原後 : \n",X_test_inverse[:3])

原始資料 : 
 [[7.3 2.9 6.3 1.8]
 [6.1 2.9 4.7 1.4]
 [6.3 2.8 5.1 1.5]]
標準化再還原後 : 
 [[7.3 2.9 6.3 1.8]
 [6.1 2.9 4.7 1.4]
 [6.3 2.8 5.1 1.5]]


#### MinMaxScaler最小最大值標準化
在 MinMaxScaler 中是給定了一個明確的最大值與最小值。每個特徵中的最小值變成了0，最大值變成了1。數據會縮放到到[0,1]之間。

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [14]:
# scaled 之後的資料最小值、最大值  
print('資料集 X 的最小值 : ', X_train.min(axis=0))
print('資料集 X 的最大值 : ', X_train.max(axis=0))

print('\nStandardScaler 縮放過後訓練集的最小值 : ', X_train_scaled.min(axis=0))
print('StandardScaler 縮放過後訓練集的最大值 : ', X_train_scaled.max(axis=0))

資料集 X 的最小值 :  [4.3 2.  1.1 0.1]
資料集 X 的最大值 :  [7.9 4.4 6.9 2.5]

StandardScaler 縮放過後訓練集的最小值 :  [0. 0. 0. 0.]
StandardScaler 縮放過後訓練集的最大值 :  [1. 1. 1. 1.]


In [15]:
X_test_scaled = scaler.transform(X_test)

print('\nStandardScaler 縮放過後測試集的最小值 : ', X_test_scaled.min(axis=0))
print('StandardScaler 縮放過後測試集的最大值 : ', X_test_scaled.max(axis=0))


StandardScaler 縮放過後測試集的最小值 :  [ 0.02777778  0.125      -0.01724138  0.04166667]
StandardScaler 縮放過後測試集的最大值 :  [0.83333333 0.83333333 0.89655172 0.95833333]


#### MaxAbsScaler絕對值最大標準化
MaxAbsScaler 與 MinMaxScaler 類似，所有數據都會除以該列絕對值後的最大值。 數據會縮放到到[-1,1]之間。

In [46]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler().fit(X)
X_scaled = scaler.transform(X)

print('\nStandardScaler 縮放過後測試集的最小值 : ', X_scaled.min(axis=0))
print('StandardScaler 縮放過後測試集的最大值 : ', X_scaled.max(axis=0)) #因為data 都 >0, 所以只會顯示1,如果有<0的資料且絕對值後是最大的, 則會出現-1


StandardScaler 縮放過後測試集的最小值 :  [0.5443038  0.45454545 0.14492754 0.04      ]
StandardScaler 縮放過後測試集的最大值 :  [1. 1. 1. 1.]


In [45]:
X_test_scaled = scaler.transform(X_test)


#### RobustScaler
可以有效的縮放帶有outlier的數據，透過Robust如果數據中含有異常值在縮放中會捨去。

In [47]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler().fit(X)
X_scaled = scaler.transform(X)

print('\nStandardScaler 縮放過後測試集的最小值 : ', X_scaled.min(axis=0))
print('StandardScaler 縮放過後測試集的最大值 : ', X_scaled.max(axis=0))


StandardScaler 縮放過後測試集的最小值 :  [-1.15384615 -2.         -0.95714286 -0.8       ]
StandardScaler 縮放過後測試集的最大值 :  [1.61538462 2.8        0.72857143 0.8       ]


In [49]:
X_test_scaled = scaler.transform(X_test)